In [1]:
import torch
import pandas as pd
import os
import sys
import numpy as np
import gym
from collections import defaultdict

dir_path = os.path.dirname(os.path.realpath('__file__'))
parent_dir = f"../src"
sys.path.insert(0, parent_dir) 

from QNetwork import QNetwork
from utils import load_trajectories, load_model, store_trajectories
from replay import play_trajectory, play_episodes

env_name = 'MountainCar-v0'
env = gym.envs.make(env_name)
discount_factor = 0.99

dir = f"{dir_path}/../data/{env_name}"

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
df = load_trajectories(env_name)
df.sort_values('episode_length').head()

,env_name,env_params,episode_length,max_reward,seed,sum_discounted_reward,sum_reward,trajectory
284,MountainCar-v0,None,88,-1.0,284,-58.705033,-88.0,"[([-0.4049334639372484, 0.0], 0, -1.0, [-0.406..."
288,MountainCar-v0,None,88,-1.0,288,-58.705033,-88.0,"[([-0.40306519008766317, 0.0], 0, -1.0, [-0.40..."
249,MountainCar-v0,None,90,-1.0,249,-59.526803,-90.0,"[([-0.40402829211419294, 0.0], 0, -1.0, [-0.40..."
281,MountainCar-v0,None,90,-1.0,281,-59.526803,-90.0,"[([-0.40820211603104967, 0.0], 0, -1.0, [-0.41..."
429,MountainCar-v0,None,94,-1.0,429,-61.121608,-94.0,"[([-0.400928104015099, 0.0], 0, -1.0, [-0.4028..."


In [5]:
episode_lengths = list(df.episode_length.sort_values())
min_len = min(episode_lengths)
max_len = max(episode_lengths)

step = (max_len - min_len) // 4
selected_trajectories = []
for val in range(min_len, max_len, step):
    closest_episode_length = min(episode_lengths, key=lambda x:abs(x - val))
    trajectory_row = df[df.episode_length == closest_episode_length].iloc[0]
    selected_trajectories.append((trajectory_row.trajectory, trajectory_row.seed))
trajectory_row = df[df.episode_length == 199].iloc[0]
selected_trajectories.append((trajectory_row.trajectory, trajectory_row.seed))

store_trajectories(env_name, selected_trajectories, env_params=None, discount=discount_factor,  filename='selected_trajectories')

In [6]:
selected_trajectories = load_trajectories(env_name, filename='selected_trajectories')
print(selected_trajectories)

         env_name env_params  episode_length  max_reward  seed  \
0  MountainCar-v0       None              88        -1.0   284   
1  MountainCar-v0       None             117        -1.0   265   
2  MountainCar-v0       None             144        -1.0   204   
3  MountainCar-v0       None             171        -1.0   163   
4  MountainCar-v0       None             199        -1.0   341   

   sum_discounted_reward  sum_reward  \
0             -58.705033       -88.0   
1             -69.145553      -117.0   
2             -76.478337      -144.0   
3             -82.068432      -171.0   
4             -86.466700      -199.0   

                                          trajectory  
0  [([-0.4049334639372484, 0.0], 0, -1.0, [-0.406...  
1  [([-0.5740905014507767, 0.0], 1, -1.0, [-0.573...  
2  [([-0.4835938327238946, 0.0], 0, -1.0, [-0.484...  
3  [([-0.4555411888264384, 0.0], 0, -1.0, [-0.457...  
4  [([-0.5207202235872288, 0.0], 0, -1.0, [-0.521...  


In [ ]:
model = load_model(env_name)
model

In [ ]:
play_episodes(env, model, n=100, render=False)